In [1]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes/')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp

test_key = 'DB318A59-25FF-499E-9A6D-783A19C346D8'

#free test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

meta_path='/home/fbuonerba/codes/meta_data/'
with open(meta_path+'top_coins.txt') as f:
    coins=json.load(f)
with open(meta_path+'symbols.txt') as ff:
    symbols=json.load(ff)

quotes=['USD','BTC']

In [ ]:
#coinAPI ohlcv data requires to use: symbols.txt#
count=0
for coi in coins:
    for quote in quotes:
        for ex in exchanges:
            sym_id=str(ex)+'_SPOT_'+str(coi)+'_'+str(quote)
            try:
                api.ohlcv_latest_data(sym_id, {'period_id': '1MTH', 'limit': 1})
                print(sym_id, 'success')
            except urllib.error.HTTPError as err:
                print(sym_id, 'error', err.code)
                count+=1

In [ ]:
#Getting data from CoinMarketCap#
web_data= urllib.request.urlopen('https://api.coinmarketcap.com/v2/ticker/')
cmc_data=web_data.read()
cmc=json.loads(cmc_data.decode('utf-8'))
cmc=cmc['data']
xxx=list(cmc.values())
#diction={}
co=0
for x in xxx:
    if x['symbol'] in coins:
        diction[x['id']]=x['symbol']
        co+=1
print(co, diction)

In [ ]:
#Getting id of coins listed on CoinMarketCap which appear in our top_coins list.
#start=100*n+1, since they limit to 100 coins per request.
web_data1= urllib.request.urlopen('https://api.coinmarketcap.com/v2/ticker/?start=1701')
cmc_data1=web_data1.read()
cmc1=json.loads(cmc_data1.decode('utf-8'))
cmc1=cmc1['data']
co1=0
xxx1=list(cmc1.values())
for x in xxx1:
    if x['symbol'] in coins:
        print(x['symbol'])
        diction[x['id']]=x['symbol']
        co1+=1
print(co1, diction, len(diction))

In [ ]:
with open(meta_path+'symbols.txt') as ll:
    symbols=json.load(ll)

In [ ]:
import os
count=0
gcount=0
S=os.listdir('/home/fbuonerba/ohlcv_data')
for s in S:
    for sym in symbols:
        if sym in s:
            gcount+=1
            with open('/home/fbuonerba/ohlcv_data/' + s,'r') as file:
                data=file.read()
            if data=='{}':
                count+=1
                print(count/gcount)
                

In [ ]:
web_data= urllib.request.urlopen('https://coinmarketcap.com/historical/20180715/')
cmc_data=web_data.read()#.decode('utf-8')

In [ ]:
cmc_data

In [ ]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(cmc_data, 'html.parser')

In [ ]:
soup

In [ ]:
So=soup.find_all('td')
So

In [ ]:
for x in So:
    print(x.contents)

In [ ]:
Vol=[]
SoVolume=soup.find_all(class_='volume')
for x in SoVolume:
    x=x.contents[0].replace('$','').replace(',','')
    x=x.replace('Low Vol','0')
    x=int(float(x))
    Vol.append(x)


In [ ]:
Supply=[]
SoSupply=soup.find_all('a',{'data-supply':True})
for x in SoSupply:
    x=x.contents[0].strip('\n').replace(',','')
    if '?' in x:
        x=float('nan')
    else:
        x=int(float(x))
    Supply.append(x)

In [ ]:
Price=[]
SoPrice=soup.find_all(class_='price')
for x in SoPrice:
    x=x.contents[0].replace('$','').replace(',','')
    x=float(x)
    Price.append(x)

In [ ]:
Syms=[]
SoSym=soup.find_all(class_="text-left col-symbol")
for x in SoSym:
    x=str(x.contents[0])
    Syms.append(x)



In [ ]:
data={}
for i in range(len(Supply)):
    data[Syms[i]]={'volume': Vol[i], 'price': Price[i], 'supply': Supply[i]}

In [ ]:
json_data=json.dumps(data)

In [ ]:
with open('/home/fbuonerba/codes/meta_data/symbols.txt') as ff:
    syms=json.load(ff)

In [ ]:
import sys
import numpy as np
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import compute_log_marketcap as lmkcap
lmkcap('MKR','USD', 1515283200,1525132800)

In [ ]:
import sys
import numpy as np
import json
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import compute_log_marketcap as clm

with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
clm('EUR','USD',1515283200,1522540800)
for coin in coins:
    x=clm(str(coin),'USD',1515283200,1522540800)
    print(coin, x)

In [2]:
from mp_functions import compute_turnover as ct
quotes=['BTC','USD']
freq=604800
beg=[1515283200+freq*t for t in range(16)]
end=[1522540800+freq*t for t in range(16)]


            print(base,quote,t)
            if [base,quote]!=['BTC','BTC'] and [base,quote]!=['BTC','USD'] and [base,quote]!=['ETH','USD']:
                sss=time.time()
                ct(base, quote, beg[t],end[t])
                print(time.time()-sss)

BTC BTC 0
ETH BTC 0
ETH BTC 1515283200 [7536466.006176567]
ETH BTC 1515888000 [7536466.006176567, 5722686.316071061]
ETH BTC 1516492800 [7536466.006176567, 5722686.316071061, 3527715.4308143836]
ETH BTC 1517097600 [7536466.006176567, 5722686.316071061, 3527715.4308143836, 3623499.4795913748]
ETH BTC 1517702400 [7536466.006176567, 5722686.316071061, 3527715.4308143836, 3623499.4795913748, 4386831.4089558]
ETH BTC 1518307200 [7536466.006176567, 5722686.316071061, 3527715.4308143836, 3623499.4795913748, 4386831.4089558, 2726739.7343385434]
ETH BTC 1518912000 [7536466.006176567, 5722686.316071061, 3527715.4308143836, 3623499.4795913748, 4386831.4089558, 2726739.7343385434, 2248133.7292801663]
ETH BTC 1519516800 [7536466.006176567, 5722686.316071061, 3527715.4308143836, 3623499.4795913748, 4386831.4089558, 2726739.7343385434, 2248133.7292801663, 1901979.6706804852]
ETH BTC 1520121600 [7536466.006176567, 5722686.316071061, 3527715.4308143836, 3623499.4795913748, 4386831.4089558, 2726739.7343

EOS BTC 1515283200 [70528852.34543173]
EOS BTC 1515888000 [70528852.34543173, 69934919.09044231]
EOS BTC 1516492800 [70528852.34543173, 69934919.09044231, 54837838.33454993]
EOS BTC 1517097600 [70528852.34543173, 69934919.09044231, 54837838.33454993, 36816385.88714906]
EOS BTC 1517702400 [70528852.34543173, 69934919.09044231, 54837838.33454993, 36816385.88714906, 38004429.294557855]
EOS BTC 1518307200 [70528852.34543173, 69934919.09044231, 54837838.33454993, 36816385.88714906, 38004429.294557855, 25368672.020384192]
EOS BTC 1518912000 [70528852.34543173, 69934919.09044231, 54837838.33454993, 36816385.88714906, 38004429.294557855, 25368672.020384192, 20424077.218018625]
EOS BTC 1519516800 [70528852.34543173, 69934919.09044231, 54837838.33454993, 36816385.88714906, 38004429.294557855, 25368672.020384192, 20424077.218018625, 19907346.39453597]
EOS BTC 1520121600 [70528852.34543173, 69934919.09044231, 54837838.33454993, 36816385.88714906, 38004429.294557855, 25368672.020384192, 20424077.21

URLError: <urlopen error [SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:645)>